In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research1/Data/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
print(os.listdir(torch_models_dir))

['block_splits_by_image_all.pth', 'block_splits_by_image_single.pth', 'eeg_14_70_std.pth', 'eeg_55_95_std.pth', 'eeg_5_95_std.pth', 'eeg_signals_raw_with_mean_std.pth']


In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
splits_all_path = os.path.join(torch_models_dir, block_splits_all)
splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)

/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_single.pth


In [7]:
splits_all = torch.load(splits_all_path)
splits_single = torch.load(splits_single_path)

In [8]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [9]:
# print(splits_single)
print(len(splits_single['splits'][0]['train']))
print(len(splits_single['splits'][0]['val']))
print(len(splits_single['splits'][0]['test']))

669
167
164


In [10]:
eeg_loaded = torch.load(eeg_dataset)

In [11]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))

print(labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[0])

3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
        [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
        [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
        ...,
        [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
        [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
        [ 

In [12]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.time_low)

20


In [13]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [14]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
             split_name,
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name = split) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [15]:
# Options
opt.classifier = "Stacked_BiLSTM"
opt.batch_size = 16
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [16]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
             opt.split_name,
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [17]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i, (input, target) in enumerate(loaders["train"]):
    if i<20:
        print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
# for i in range(0, 40):
#     eeg, label_val = splitter["val"][i]
#     eeg, label_train = splitter["train"][i]
#     print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [498, 125, 125]
1: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
2: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
3: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
4: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
5: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
6: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
7: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
8: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
9: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
10: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
11: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
12: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
13: Target size: torch.Size([16]); input size:

In [18]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_Stacked_BiLSTM(
  (stacked_bilstm): LSTM(128, 128, num_layers=6, batch_first=True, bidirectional=True)
  (output1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_Stacked_BiLSTM                [1, 40]                   --
├─LSTM: 1-1                              [1, 440, 256]             2,240,512
├─Linear: 1-2                            [1, 128]                  32,896
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 2,278,568
Trainable params: 2,278,568
Non-trainable params: 0
Total mult-adds (M): 985.86
Input size (MB): 0.23
Forward/backward pass size (MB): 0.90
Params size (MB): 9.11
Estimated Total Size (MB): 10.24

In [19]:
model_path = (   opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) + "-" + "6layers" )
print(model_path)

Stacked_BiLSTM-440-128-6layers


In [20]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='Stacked_BiLSTM')


In [ ]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

Epoch 1
Train Batch 100 (every 100 batch): Loss=3.6593; accuracy=0.0250
Train Batch 200 (every 100 batch): Loss=3.6804; accuracy=0.0259
Train Batch 300 (every 100 batch): Loss=3.6826; accuracy=0.0250
Train Batch 400 (every 100 batch): Loss=3.6758; accuracy=0.0267
Epoch 1 summary: train_loss: 3.6903 | train_acc: 0.0267 | val_loss: 3.6998 | val_acc: 0.0243
Epoch 2
Train Batch 100 (every 100 batch): Loss=3.6807; accuracy=0.0312
Train Batch 200 (every 100 batch): Loss=3.6914; accuracy=0.0325
Train Batch 300 (every 100 batch): Loss=3.6916; accuracy=0.0312
Train Batch 400 (every 100 batch): Loss=3.6722; accuracy=0.0295
Epoch 2 summary: train_loss: 3.6878 | train_acc: 0.0305 | val_loss: 3.7033 | val_acc: 0.0240
Epoch 3
Train Batch 100 (every 100 batch): Loss=3.7155; accuracy=0.0281
Train Batch 200 (every 100 batch): Loss=3.7035; accuracy=0.0291
Train Batch 300 (every 100 batch): Loss=3.7097; accuracy=0.0296
Train Batch 400 (every 100 batch): Loss=3.6903; accuracy=0.0291
Epoch 3 summary: train

Train Batch 100 (every 100 batch): Loss=2.1188; accuracy=0.2062
Train Batch 200 (every 100 batch): Loss=2.5336; accuracy=0.2069
Train Batch 300 (every 100 batch): Loss=2.2547; accuracy=0.2054
Train Batch 400 (every 100 batch): Loss=2.3587; accuracy=0.2011
Epoch 24 summary: train_loss: 2.1702 | train_acc: 0.2017 | val_loss: 2.3996 | val_acc: 0.1555
Epoch 25
Train Batch 100 (every 100 batch): Loss=1.8461; accuracy=0.2369
Train Batch 200 (every 100 batch): Loss=2.3798; accuracy=0.2241
Train Batch 300 (every 100 batch): Loss=1.7655; accuracy=0.2206
Train Batch 400 (every 100 batch): Loss=2.2577; accuracy=0.2203
Epoch 25 summary: train_loss: 2.0904 | train_acc: 0.2165 | val_loss: 2.3838 | val_acc: 0.1766
Epoch 26
Train Batch 100 (every 100 batch): Loss=2.1674; accuracy=0.2400
Train Batch 200 (every 100 batch): Loss=2.0193; accuracy=0.2400
Train Batch 300 (every 100 batch): Loss=1.6424; accuracy=0.2410
Train Batch 400 (every 100 batch): Loss=2.2197; accuracy=0.2402
Epoch 26 summary: train_lo

Train Batch 100 (every 100 batch): Loss=1.2018; accuracy=0.5756
Train Batch 200 (every 100 batch): Loss=1.1033; accuracy=0.5778
Train Batch 300 (every 100 batch): Loss=1.1589; accuracy=0.5646
Train Batch 400 (every 100 batch): Loss=0.8275; accuracy=0.5650
Epoch 47 summary: train_loss: 1.1255 | train_acc: 0.5655 | val_loss: 3.3174 | val_acc: 0.2077
Epoch 48
Train Batch 100 (every 100 batch): Loss=1.0397; accuracy=0.6175
Train Batch 200 (every 100 batch): Loss=0.6639; accuracy=0.5909
Train Batch 300 (every 100 batch): Loss=1.3855; accuracy=0.5867
Train Batch 400 (every 100 batch): Loss=1.1090; accuracy=0.5920
Epoch 48 summary: train_loss: 1.0854 | train_acc: 0.5869 | val_loss: 3.4144 | val_acc: 0.2243
Epoch 49
Train Batch 100 (every 100 batch): Loss=1.0501; accuracy=0.5950
Train Batch 200 (every 100 batch): Loss=1.4079; accuracy=0.6019
Train Batch 300 (every 100 batch): Loss=0.9143; accuracy=0.5954
Train Batch 400 (every 100 batch): Loss=0.6530; accuracy=0.5745
Epoch 49 summary: train_lo

Train Batch 100 (every 100 batch): Loss=0.6353; accuracy=0.8100
Train Batch 200 (every 100 batch): Loss=0.6775; accuracy=0.7947
Train Batch 300 (every 100 batch): Loss=0.8788; accuracy=0.7919
Train Batch 400 (every 100 batch): Loss=1.1105; accuracy=0.7834
Epoch 70 summary: train_loss: 0.6107 | train_acc: 0.7720 | val_loss: 4.5437 | val_acc: 0.1976
Epoch 71
Train Batch 100 (every 100 batch): Loss=0.5074; accuracy=0.7887
Train Batch 200 (every 100 batch): Loss=0.9475; accuracy=0.7903
Train Batch 300 (every 100 batch): Loss=0.4285; accuracy=0.7752
Train Batch 400 (every 100 batch): Loss=0.5376; accuracy=0.7784
Epoch 71 summary: train_loss: 0.6325 | train_acc: 0.7718 | val_loss: 4.8072 | val_acc: 0.1990
Epoch 72
Train Batch 100 (every 100 batch): Loss=0.2289; accuracy=0.7856
Train Batch 200 (every 100 batch): Loss=0.4670; accuracy=0.7862
Train Batch 300 (every 100 batch): Loss=0.3773; accuracy=0.7983
Train Batch 400 (every 100 batch): Loss=0.4545; accuracy=0.7933
Epoch 72 summary: train_lo

In [ ]:
# val =accuracy_val
# test = accuracy_test

# print("Validation accuracy: ", val)
# print("Test accuracy: ", test)

In [ ]:
print(accuracy_val)